# Markov Chain Monte Carlo with Couplings and Approximate Bayesian Computation, parallelized, univariate mean Gaussian 

## Modules importing

In [1]:
# system modules
import os
import time

# mathematical and statistical modules
import math
import numpy as np
import pandas as pd
import scipy.stats as sstat
#from mpmath import mp

# tensorflow modules
import tensorflow_probability.substrates.numpy as tfp
import tensorflow_probability.substrates.numpy.distributions as tfd
import tensorflow.random as tfr

# visualization and plotting modules
import matplotlib.pyplot as plt
import seaborn as sns

# parallelization modules 
from multiprocessing import Process, Queue, SimpleQueue

2022-01-10 11:06:09.273285: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-01-10 11:06:09.273316: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Settings and data

### Parameters

In [2]:
#data
number_of_data     = 100
observations_mean  = np.array([10.])
observations_sd    = np.sqrt(3.) 

#prior
prior_mean = np.array([8.])
prior_sd = 2

#mcmc setting
#x_start = np.array([14., 23.])
#y_start = np.array([ 8., 16.])
iterations = 500
burnout = 100

# coupling tolerance
chains_matching_tolerance = 1e-3

# time averaged estimator parameter
test_function_h = lambda x: x

## Generation of data
Data are generated and placed in the variable "observations_y"; those data will be used in the likelihood function.

ATTENZIONE: RISPETTO AI CODICI PRECEDENTI LE observations_y SONO TRASPOSTE!

In [3]:
observations_y = np.array(
    np.random.normal(
        loc=observations_mean, 
        scale=(observations_sd**2), 
        size=number_of_data
    )
)
observations_y.shape

(100,)

### Distributions functions

Likelihood functions: the first given the mean calculate the probability (pdf) of a given value, the second return a number of samples, as much as value, the third is the likelihood function.

In [4]:
def single_likelihood_prob(mean, value):
    return sstat.norm(mean, (observations_sd**2)).pdf(value)

In [5]:
def single_likelihood_sample(mean, value):
    array = sstat.norm(mean, (observations_sd**2)).rvs(value)
    return np.transpose(array)

In [6]:
def likelihood(mu):
    tau = 1/(observations_sd**2)
    return np.exp(  
        -(tau/2)*( 
            number_of_data * (mu - observations_mean)**2 
            + np.sum( (np.transpose(observations_y) - observations_mean)**2, axis = 0) 
            )  
        ) * 10.**10 #this coefficient is to avoid annihilation - it's a rescale

Prior function and prior sampler (only one sample for each call)

In [7]:
def prior(value):
    return sstat.norm(
        prior_mean, 
        (prior_sd**2)
        ).pdf(value)

In [8]:
def prior_sampling():
    return sstat.norm(
        prior_mean, 
        (prior_sd**2)
        ).rvs(1)

Target distribution

In [9]:
def target(value):
    return likelihood(value)*prior(value)

Proposal probability and sample (one for each call), given the mean

In [10]:
def proposal_prob(mean, value):
    return sstat.norm(mean, (observations_sd**2)).pdf(value)

In [11]:
def proposal_sample(mean):
    return sstat.norm(mean, (observations_sd**2)).rvs(1)

Kernel for ABC

In [12]:
def kernel(u):
    return 1/(np.sqrt(2*math.pi))*np.exp(-1/2*u**2)

In [13]:
def K_h(u,h):
    return kernel(u/h)/h

Summary statistics

In [14]:
def S1(y):
    quantiles=np.empty(9)
    for i in range(9):
        quantiles[i] = np.quantile(
            y,
            0.1*(i+1)
        ) 
    return quantiles

In [15]:
def S2(y):
      return np.mean(y)

## Functions

### Maximal coupling
The algorithm:
1)  
    1) sample $X \sim p$
    2) sample $W|X \sim \mathcal{U}\{[0,p(X)]\}$ \
    if $W\leq q(X)$ then output $(X,X)$
2)  otherwise
    1) sample $Y^\star \sim q$
    2) sample $W^\star | Y^\star \sim \mathcal{U}\{[0, q(Y^\star)]\}$ \
    until $W^\star > p(Y^\star)$ and output $(X,Y^\star)$
   

In [16]:
def maximal_coupling(x,y):     
    dimensionality = 1

    #sampling of X - point 1.A
    #the new candidate is sampled from a normal centered in the given value
    #x_candidate = np.random.multivariate_normal(mean=x, cov=np.eye(dimensionality), size=1)
    x_candidate = sstat.multivariate_normal(x,np.eye(dimensionality)).rvs(1)
    
    #sampling of W|X - point 1.B
    #evaluate pdf centered in x with scale 1 in x_cadidate
    pX = sstat.multivariate_normal(x,np.eye(dimensionality)).pdf(x_candidate)
    W = tfd.Uniform(0,pX).sample()
    #sampling from uniform is inside the cycle on parameters 
    
    # evaluation of qx - point 1 IF
    qX = sstat.multivariate_normal(y, np.eye(dimensionality)).pdf(x_candidate)
    
    if W < qX:
        x = x_candidate
        y = x_candidate
    else:
        while True:
            #sampling of Y* - point 2.A
            y_candidate = sstat.multivariate_normal(y,np.eye(dimensionality)).rvs(1)

            #sampling of W* - point 2.B
            qY = sstat.multivariate_normal(y, np.eye(dimensionality)).pdf(y_candidate)
            W =  tfd.Uniform(0,qY).sample()

            # evaluation of py - point 2
            pY = sstat.multivariate_normal(x,np.eye(dimensionality)).pdf(y_candidate)
            if W > pY :
                break

        x = x_candidate
        y = y_candidate

    return (x, y)

### Maximal coupling for ABC

In [17]:
def maximal_coupling_abc(x,y):     
    dimensionality = 1

    #sampling of X - point 1.A
    #the new candidate is sampled from a normal centered in the given value
    #x_candidate = np.random.multivariate_normal(mean=x, cov=np.eye(dimensionality), size=1)
    x_candidate = single_likelihood_sample(x, 1)
    
    #sampling of W|X - point 1.B
    #evaluate pdf centered in x with scale 1 in x_cadidate
    pX = single_likelihood_prob(x, x_candidate)
    W = tfd.Uniform(0,pX).sample()
    #sampling from uniform is inside the cycle on parameters 
    
    # evaluation of qx - point 1 IF
    qX = single_likelihood_prob(y, x_candidate)
    if W < qX:
        x = x_candidate
        y = x_candidate
    else:
        while True:
            #sampling of Y* - point 2.A
            y_candidate = single_likelihood_sample(y, 1)

            #sampling of W* - point 2.B
            qY = single_likelihood_prob(y, y_candidate)
            W =  tfd.Uniform(0,qY).sample()

            # evaluation of py - point 2
            pY = single_likelihood_prob(x, y_candidate)
            if W > pY :
                break

        x = x_candidate
        y = y_candidate

    return (x, y)

### Metropolis Hastings - with ABC
THE COMPLETE ALGORITM HAS NOT BEEN WRITTEN HERE
The algorithm:
1) sample $(X^\star, Y^\star) | (X_t, Y_{t-1})$ from a maximal coupling of $q(X_t, \cdot)$ and $q(Y_{t-1}, \cdot)$
2) sample $U \sim \mathcal{U}([0,1])$,
3) if $$ U
 				\leq \min\bigg \{
 				1,
 				\frac{ \pi(X^\star)q(X^\star,X_t)}{
 					\pi(X_t)q(X_t, X^\star)}
 				\bigg \} $$ then $X_{t+1} = X^\star$; otherwise $X_t = X_{t-1}$;
4) if $$ U	\leq \min\bigg \{ 
 				1,
 				\frac{ \pi(Y^\star)q(Y^\star,Y_t)}{
 					\pi(Y_t)q(Y_t, Y^\star)}
 				\bigg \} $$
    then $Y_{t+1} = Y^\star$; otherwise $Y_t = Y_{t-1}$.
   

In [18]:
def metropolis_hastings(iterations, observations_y, S, h):
    
    x = prior_sampling()
    y = prior_sampling()

    
    dimensionality = 1
    
    x = np.array(x)  # problema di compatibilità
    y = np.array(y)
    
    # initializing empty np arrays
    X = np.empty([dimensionality, iterations])
    Y = np.empty([dimensionality, iterations])
    
    #ABC
    size_abc = 100
    
    X_abc = np.empty([dimensionality, size_abc])
    Y_abc = np.empty([dimensionality, size_abc])
    
    observations_s = S(observations_y)

    for j in range (size_abc):
        (X_abc[:,j], Y_abc[:,j]) = maximal_coupling_abc(x,y)
    
    sx = S(X_abc)
    sy = S(Y_abc)
    
        
    while K_h(np.linalg.norm(sx - observations_s),h) <= 0:
        x = prior_sampling()
        sx = S( single_likelihood_sample(x, size_abc) )

    while K_h(np.linalg.norm(sy - observations_s),h) <= 0:
        y = prior_sampling()
        sy = S(single_likelihood_sample(y, size_abc))
        
    x = np.array(x)  # problema di compatibilità
    y = np.array(y)

    for i in range(iterations):
        # sample from maximal coupling - point 1
        (x_candidate, y_candidate) = maximal_coupling(x, y) #da qui deve uscire già multivariato
        x_candidate = np.array(x_candidate)
        y_candidate = np.array(y_candidate)
        
        #ABC start
        #X_abc = np.empty([dimensionality, size_abc])
        #Y_abc = np.empty([dimensionality, size_abc])
        
        for j in range(size_abc):
            (X_abc[:,j], Y_abc[:,j]) = maximal_coupling_abc(x_candidate, y_candidate)
        
        sx_candidate = S(X_abc) 
        sy_candidate = S(Y_abc)
        
           
        #print('lmao')
        #print(X_prob_abc)
        #print(Y_prob_abc)
        # sample from a uniform - point 2
        u = tfd.Uniform(0,1).sample()
        
        X_prob_abc = min(
                [1.,(
                     K_h(np.linalg.norm(sx_candidate - observations_s),h)*
                     prior(x_candidate)
                 )/(
                     K_h(np.linalg.norm(sx - observations_s),h)*
                     prior(x)
                )]
            )

        Y_prob_abc = min(
                [1.,(
                    K_h(np.linalg.norm(sy_candidate - observations_s),h)*
                    prior(y_candidate)
                )/(
                    K_h(np.linalg.norm(sy - observations_s),h)*
                    prior(y)
                )]
            )

        # points 3 and 4 are to be executed on all the parameters which determine the dimensionality
        #for param in range(dimensionality):
        # check if the sampling of X is accepted or to consider the previous - point 3
        

        if u < X_prob_abc :
            x = x_candidate
            sx = sx_candidate

        X[:,i]=x

        if u < Y_prob_abc :
            y = y_candidate
            sy = sy_candidate

        Y[:, i] = y
          
    return X, Y

### Time Averaged estimator
The algorithm:
1) draw $X_0$ and $Y_0$ from an initial distribution $\pi_0$ and draw $X_1 \sim P(X_0, \cdot)$;
2) set $t=1$: while $t<\max\{m,\tau\}$ and:
    1) draw $(X_{t+1}, Y_t)\sim \bar P \{(X_t, Y_{t-1}), \cdot \}$; 
    2) set $t \leftarrow t+1$;
3) compute the time-averaged estimator:
	$$ H_{k:m}(X,Y)
	 		= \frac{1}{m-k+1}\sum_{l=k}^{m}h(X_l) 
	 		+ \sum_{l=k+1}^{\tau -1}\min(1, \frac{l-k}{m-k+1})\{h(X_l)-h(Y_{l-1})\} .
	 		$$

In [19]:
def time_averaged_estimator(red_chain, blue_chain, iterations, burnout, meeting_time):
    #this function calculates only the formula at point 3
    
    #first addend
    first_sum = 0
    for i in range(burnout, iterations):
        first_sum += test_function_h(red_chain[i])
    
    #second addend
    second_sum = 0
    if meeting_time-1 > burnout + 1:
        for i in range(burnout + 1, meeting_time):
            # calculing the coefficient, and the min
            coef = (1-burnout)(iterations - burnout + 1)
            if coef > 1:
                coef = 1
            second_sum += coef * (test_function_h(red_chain[i])- test_function_h(blue_chain[i]))
            
    return (1/(iterations - burnout + 1))*first_sum + second_sum
    

### Parallelized operations
This function is given to each available processor

In [20]:
def parallelized_operations(iterations, observations_y, S, h, burnout, chains_matching_tolerance, q_chain, q_meeting_time, q_time_averaged_estimator):
    # setting a random seed - otherwise multiprocessing has the same seed and so they end up in the same chains!
    np.random.seed(int(os.getpid() * time.time()) % 123456789 )
    tfr.set_seed(int(os.getpid() * time.time()) % 123456789 )
    
    #running metropolis hastings and obtaining two chains
    (red_chain, blue_chain) = metropolis_hastings(iterations, observations_y, S, h)
    
    #get the dimensionality from one of the chains
    dimensionality = red_chain.shape[0]

    #finding the meeting time
    meeting_time = iterations + 2
    for i in range(0, iterations):
        if (abs(red_chain[:,i]-blue_chain[:,i]).all() < chains_matching_tolerance) & \
            (abs(red_chain[:,i]-blue_chain[:,i]).all() < chains_matching_tolerance) & \
            (abs(red_chain[:,i]-blue_chain[:,i]).all() < chains_matching_tolerance):
            meeting_time = i
            break
    
    #checking if the chains met
    if meeting_time == iterations+2:
        print('Matching did not occurred!')
        print(meeting_time)
        return
    
    #checking if the chains met before burnout
    if meeting_time >= burnout:
        print('Meeting time occurs after burnout iterations!')
        print(meeting_time)
        return
    
    time_averaged_estimation = np.empty(dimensionality)
    
    for param in range(dimensionality):
        time_averaged_estimation[param] = time_averaged_estimator(
            red_chain[param,], 
            blue_chain[param,], 
            iterations, burnout, meeting_time
        )
    #print(red_chain)
    q_chain.put( red_chain )
    q_meeting_time.put( meeting_time )
    q_time_averaged_estimator.put( time_averaged_estimation )
    
    return

    

## Execution

### Multiprocessing settings

In [21]:
# this returns the number of available processors
num_processes = os.cpu_count()
num_processes

8

In [22]:
# this set the number of iteration, it should be a multiple of the number of available processors
num_batches_set = 16
num_batches = num_batches_set

In [23]:
# this initialize a shared memory between processors
# note of Queue(maxsize = 0)
q_chain = SimpleQueue()
q_meeting_time = SimpleQueue()
q_time_averaged_estimator = SimpleQueue()

### Multiprocessing execution

In [24]:
dimensionality = 1 # observations_y.shape[0]

samplings = np.empty([dimensionality, 0])
time_averaged_estimation_sum = 0
meeting_times = np.empty(0)
correct_chains = 0

while num_batches > 0:
    
    print("There are " + str(num_batches) + " left")
    
    if num_batches >= num_processes:
        current_number_processes = num_processes
    else:
        current_number_processes = num_batches
    
    processes = []
    for batch in range(current_number_processes):
        p = Process(
            target = parallelized_operations, 
            args = (iterations,  
                    observations_y, S2, 0.2,
                    burnout, 
                    chains_matching_tolerance, 
                    q_chain, q_meeting_time, q_time_averaged_estimator)
        )
        processes.append(p)

    num_batches -= current_number_processes
    
    # Starting the processes in a batch
    for p in processes:
        p.start()
        time.sleep(0.1)

    # This assures that each process in the batch is complete
    for p in processes:
        p.join()

    #unfolding queues
    while not q_chain.empty():
        red_chain = q_chain.get()
        samplings = np.append(
            samplings,
            red_chain[:,burnout :],
            axis = 1
        )


        meeting_time = q_meeting_time.get()
        time_averaged_estimation = q_time_averaged_estimator.get()
        
        if meeting_time <= burnout: #this to avoid a biased time averaged estimator
            time_averaged_estimation_sum += time_averaged_estimation
            correct_chains += 1
        
        meeting_times = np.append(
            meeting_times,
            meeting_time
        )
    
del q_chain, q_meeting_time, q_time_averaged_estimator
print('All processes are complete!')

There are 16 left
There are 8 left
All processes are complete!


## Output and plotting

### Correct executions infos

In [25]:
print(meeting_times)
print(len(meeting_times))
print(num_batches_set-len(meeting_times))

[ 5.  3.  6.  9. 15. 14. 28. 31.  8. 15.  7. 16. 11. 15. 36. 34.]
16
0


In [26]:
samplings.size

6400

### Time averaged estimator

In [27]:
time_averaged_estimation_mean = time_averaged_estimation_sum/correct_chains
time_averaged_estimation_mean

array([9.7922513])

In [28]:
samplings.shape

(1, 6400)

In [29]:
plt.plot(samplings[0,:])

(array([  6.,   6.,  14.,  26.,  23.,  45.,  50.,  30.,  76.,  65., 146.,
        110., 123., 255., 218., 220., 295., 405., 439., 373., 338., 369.,
        372., 255., 341., 396., 210., 263., 185., 144.,  79., 121.,  77.,
        110.,  30.,  45.,  26.,  12.,  51.,   6.,   4.,   0.,   0.,   0.,
          0.,   0.,   0.,   0.,   0.,  41.]),
 array([ 8.71192898,  8.76438553,  8.81684209,  8.86929864,  8.92175519,
         8.97421174,  9.0266683 ,  9.07912485,  9.1315814 ,  9.18403795,
         9.23649451,  9.28895106,  9.34140761,  9.39386416,  9.44632071,
         9.49877727,  9.55123382,  9.60369037,  9.65614692,  9.70860348,
         9.76106003,  9.81351658,  9.86597313,  9.91842969,  9.97088624,
        10.02334279, 10.07579934, 10.12825589, 10.18071245, 10.233169  ,
        10.28562555, 10.3380821 , 10.39053866, 10.44299521, 10.49545176,
        10.54790831, 10.60036487, 10.65282142, 10.70527797, 10.75773452,
        10.81019108, 10.86264763, 10.91510418, 10.96756073, 11.02001728,
 

In [31]:
plt.hist(samplings[0,:], bins = 50)

### Plots